In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, pool_size, dropout):
        super().__init__()
        self.conv = nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, padding='same')
        self.mp = nn.MaxPool1d(kernel_size=pool_size, stride=pool_size)
        self.do = nn.Dropout(dropout)

    def forward(self, x):
        x = F.relu(self.conv(x))
        
        # print(x.shape)
        
        x = self.mp(x)
        x = self.do(x)
        return x

class DeepGXP(nn.Module):
    """DeepGXP model derived from DanQ.

    conv_out_dim = 320
    conv_kernel_sizes = [10, 15]
    pool_size = 4
    lstm_hidden_dim = 320
    fc_hidden_dim = 64
    dropout1 = 0.2
    dropout2 = 0.5

    """
    def __init__(self, conv_out_dim= 512, conv_kernel_sizes=[9, 15], pool_size=1, lstm_hidden_size= 320, dropout1=0.2, dropout2=0.5):  
        super().__init__()
        # Default
        ## lstm_hidden_size= 320
        
        out_channels_each = int(conv_out_dim / len(conv_kernel_sizes))  # 256
                
        self.conv_blocks1 = nn.ModuleList([
            ConvBlock(40, out_channels_each, k, pool_size, dropout1) for k in conv_kernel_sizes
        ])
        self.bi_lstm = nn.LSTM(input_size=conv_out_dim, hidden_size=lstm_hidden_size, batch_first=True, bidirectional=True)

        out_channels_each = int(lstm_hidden_size / len(conv_kernel_sizes))
        self.conv_blocks2 = nn.ModuleList([
            ConvBlock(lstm_hidden_size * 2, out_channels_each, k, pool_size, dropout1) for k in conv_kernel_sizes
        ])
        self.do2 = nn.Dropout(0.5)
        
        self.linear1 = nn.Linear(lstm_hidden_size * 2, 64)
        self.linear2 = nn.Linear(64, 3)
        
        
        # self.linear1 = nn.Linear(lstm_hidden_size * 2 * 110 // 2, 64)
        # self.linear1 = nn.Linear(lstm_hidden_size * 2 * 298, 64)
        # self.linear2 = nn.Linear(64, 64)
        # self.linear3 = nn.Linear(64, 1)

        self.lstm_hidden_size = lstm_hidden_size
    
    def forward(self, x):
        """Expects input shape x: bsz x 4 x L
        where max sequence length = L
        """
        bsz = x.size(0) # batch size
        # h0 = torch.zeros(2, bsz, self.lstm_hidden_size).to(x.device)
        # c0 = torch.zeros(2, bsz, self.lstm_hidden_size).to(x.device)
        ## do not need to create Initial zero tensor

        conv_outputs = []
        for conv in self.conv_blocks1:            
            conv_outputs.append(conv(x))
        x = torch.cat(conv_outputs, dim=1)
        
        print("여기까지 문제없다!") # [32, 512, 298], [B, Ch, L]
        print(x.shape)

        # x, (h, c) = self.bi_lstm(x.permute(0, 2, 1), (h0, c0))
        x, (h, c) = self.bi_lstm(x.permute(0, 2, 1)) # [B, L, Ch]
        
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.do2(x)
        
        x = x.permute(0, 2, 1)
        
        # x = x.permute(0, 2, 1) # bsz x 320 x L

        # conv_outputs = []
        # for conv in self.conv_blocks2:
        #     conv_outputs.append(conv(x))
        # x = torch.cat(conv_outputs, dim=1).permute(0, 2, 1)
        
        # print("여기까지 문제없다!")
        # print(x.shape) # [32, 298, 320], [B, L, Ch]

        # x = self.do2(x)

        # x = F.relu(self.linear1(x.reshape(bsz, -1)))
        # x = F.relu(self.linear2(x))
        # x = self.linear3(x)

        return x

In [ ]:
model = DeepGXP()

x = torch.randn([32, 40, 298]) # batch, ???, Length
output = model(x)

output.shape

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [21]:
in_channels= 4
out_channels= 6
kernel_size= 3

conv1d_layer1 = nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size)

In [ ]:
input_data = torch.rand(1, 4, 10)

output = conv1d_layer1(input_data)
output.shape

In [1]:
cd /Data1/hmd2/notebooks_th/Hmd_github/neural_networks

/Data1/hmd2/notebooks_th/Hmd_github/neural_networks


In [2]:
ls

CRNN.py  GRU_hidden.py  MHA_LSTM_hidden.py  __init__.py  __pycache__/


In [10]:
from MHA_LSTM_hidden import MHA_LSTM_hidden

model = MHA_LSTM_hidden()

In [14]:
import torch 

input = torch.randn(4, 40 ,298)
seq_pred, murmur_pred = model(input)

In [17]:
seq_pred.shape, murmur_pred.shape

(torch.Size([4, 3, 298]), torch.Size([4, 2]))